In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
# 예외처리
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException, NoSuchWindowException, WebDriverException

import requests
import re
import time
import pandas as pd
from bs4 import BeautifulSoup

# 사용할 csv 불러오기
* 파일 path 각자에 맞게 설정해줘야함.

In [ ]:
# 각자 맞게 파일 path 설정
file = '../data/구석구석dataset_수정.csv'

In [ ]:
# csv 파일 읽기
구석구석_raw = pd.read_csv(file, encoding='cp949', index_col=0)
구석구석_df = 구석구석_raw.copy()

1. 네이버 모바일에서 검색하기
2. 상단의 **지도** 탭 클릭하기
3. 검색한 관광지명에 맞는 구이름 찾고 해당 관광지 선택하기
4. **리뷰** 탭 클릭하기
5. **이런 점이 좋았어요** 찾고 저장하기
    * **이런 점이 좋았어요**가 있다면
        * **좋은 점** 목록이 있는지 확인하기
            1) **공개됩니다**라는 글자가 있다면 목록이 공개되지 않아 내용을 가져올 수 없음.
                * 관광지명과 해당 주소 list 저장. 리뷰내용, 선택한 사람수, 총 사람 수는 None으로 list 저장
            2) 목록이 있다면 내용 저장.
                * 관광지명, 주소, 리뷰내용, 선택한 사람 수, 총 사람 수
    * 없다면
        * 관광지명, 주소 저장. 리뷰내용, 선택 사람 수, 총 사람 수는 None

# 네이버 리뷰 크롤링 함수

## 지도 탭에서 선택하기 함수

In [ ]:
# 지도 선택하기 함수
def click_지도():
    tabs = driver.find_elements(By.CSS_SELECTOR, 'a.tab')
    for i, tab in enumerate(tabs):
        try:
            if '지도' in tab.text:
                css = f'#_sch_tab > div > div:nth-child({i+1}) > a'
                button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))
                print(f'{i+1}번째에 지도 탭있음.')
                break
            else:
                continue
        except:
            print('지도 탭 없음.')

    return button

## 관광지 선택하기 함수
* 여기에서 **관광지 주소**를 **location**이라는 변수에 저장함.

In [ ]:
# 지도에서 원하는 관광지 선택하기 함수
def click_관광지(x):
    # 해당 관광지의 구이름
    loca_short = x
    # 원하는 관광지명의 시군구명이 같은 항목 선택하기
    # <a href="#" class="item_address _btnAddress" data-nclicks="plc_ntl.roadon" data-cid="11796387" data-rank="1">서울특별시 광진구 능동로 216</a>
    addrs = driver.find_elements(By.CLASS_NAME, 'item_address._btnAddress')
    # 반복문 돌려서 원하는 구 이름 찾으면 선택하기
    for i, addr in enumerate(addrs):
        try:
            # 구이름이랑 맞으면
            if loca_short in addr.text:
                # 도로명주소 가져오기
                location = addr.text.split('\n')[1]
                print(location)
                # 찾고 있는 관광지 클릭 준비
                loc_css = f'#ct > div.search_listview._content._ctList > ul > li:nth-child({i+1}) > div.item_info > a.a_item.a_item_distance._linkSiteview'
                # loc_xpath = f'//*[@id="ct"]/div[2]/ul/li[{i+1}]/div[1]/a[2]'
                # //*[@id="ct"]/div[2]/ul/li[2]/div[1]/a[2]
                # #ct > div.search_listview._content._ctAddress > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview
                button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, loc_css)))
                print(f'{i+1}번째에 해당 관광지 있음.')
                break
            else:
                continue
        except:
            print('해당 관광지 없음.')

    return button, location

## 리뷰 탭 선택하기 함수

In [ ]:
# 카테고리 추출 후, 리뷰 탭 선택하기 함수
def click_리뷰():
    # 카테고리 선택
    # <span class="lnJFt">카페,디저트</span>
    category = driver.find_element(By.CLASS_NAME, 'lnJFt').text
    # <a href="/place/11796387/review" role="tab" class="tpj9w _tab-menu" aria-selected="false" title="" id="" style="width: 158px;"><span class="veBoZ">리뷰</span></a>
    tabs = driver.find_elements(By.CSS_SELECTOR, 'a.tpj9w._tab-menu > span.veBoZ')
    for i, tab in enumerate(tabs):
        try:
            if '리뷰' in tab.text:
                # #app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(5)
                css = f'#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child({i+1})'
                button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css)))
                print(f'{i+1}번째에 리뷰 탭있음.')
                break
            else:
                continue
        except:
            print('리뷰 탭 없음.')

    return button, category

## 이런 점이 좋았어요 찾고 좋은점 목록 찾고 저장하기 함수
* 여기에서 **주소**, **총 참여 인원 수** 추출 가능

In [ ]:
# '이런 점이 좋았어요' 찾고 저장하는 함수
def find_save_좋은점(list_):
    # 이런 점이 좋았어요 확인
    good_point = driver.find_element(By.CLASS_NAME, 'place_section_header')
    if '이런 점이 좋았어요' in good_point.text:
        print('"이런 점이 좋았어요" 있음.')
        # '이런 점이 좋았어요' 목록 확인
        # 최대 10번 클릭 시도
        for i in range(10):
            try:
                button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'dP0sq')))
                button.click()
                print(f"{i+1}번째 버튼 클릭 성공")

            # 버튼이 더 이상 존재하지 않으면 반복 중단
            except TimeoutException:
                print(f"{i+1}번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.")
                break

        try:
            # 좋은 점 개수 세기
            num = len(driver.find_elements(By.CLASS_NAME, 'MHaAm'))
            # <div class="zXXsw"><em>10</em>명 이상 참여하면<br>방문 키워드 통계가 공개됩니다!</div>
            blind = driver.find_element(By.CLASS_NAME, 'zXXsw')

            if '공개됩니다' in blind.text:
                print('리뷰수가 너무 적어 목록이 생성되지 않았습니다.')
                list_.append({'관광지명' : spot, '주소' : location, '목록' : category, '리뷰내용': None , '리뷰수': None, '참여인원수' : None})

        except NoSuchElementException:
            # 좋은 점 가져오기
            # 리뷰 개수만큼 반복
            for n in range(0, num):
                # 리뷰 내용
                contents = driver.find_elements(By.CLASS_NAME, 't3JSf')[n].text
                # 선택한 수
                review_num = driver.find_elements(By.CLASS_NAME, 'CUoLy')[n].text.split('\n')[1]
                # 참여인원수 추출
                # <span class="mholG">263명 참여</span>
                reviewer = driver.find_element(By.CLASS_NAME, 'mholG').text.split()[0]
                # 리스트에 관광지명, 도로명주소, 리뷰내용과 리뷰수를 딕셔너리로 담기
                list_.append({'관광지명' : spot, '주소' : location, '목록' : category, '리뷰내용': contents , '리뷰수': review_num, '참여인원수' : reviewer})
                # 저장 완료 표시
                print(f'----- {n+1}번째 좋은 점 저장 -----')

    else:
        print('"이런 점이 좋았어요" 없음.')
        list_.append({'관광지명' : spot, '주소' : location, '목록' : category, '리뷰내용': None , '리뷰수': None, '참여인원수' : None})

    return list_

# **리뷰 찾기 코드**

In [ ]:
# 에러 안 난 결과 리스트
results = []
# 에러 난 결과 리스트
cant_find = []

# 사용할 DF
df = 구석구석_df[2102:2104]

# 웹 드라이버 실행
driver = webdriver.Chrome()
# 웹 드라이버 최대화
driver.maximize_window()

# 반복문으로 관광지 돌리기
for i in range(df.shape[0]):
    try:
        # 관광지명 가져오기
        spot = df.iloc[i, 0]
        # 구 이름 가져오기
        loca_short = df.iloc[i, 1].split()[1]
        print('-'*20, i+1, ':', spot, '-'*20)

        # spot 찾기
        driver.get(f"https://m.search.naver.com/search.naver?sm=mtp_sly.hst&where=m&query={spot}")
        time.sleep(6)

        # 지도 탭 찾기
        search_button = click_지도()
        if search_button != None:
            # 지도 선택하기
            search_button.click()
            time.sleep(3)

            # 해당 관광지 찾기
            search_button, location = click_관광지(loca_short)
            if search_button != None:
                # 해당 관광지 선택하기
                search_button.click()
                time.sleep(3)

                # 리뷰 선택하기
                search_button, category = click_리뷰()
                if search_button != None:
                    # 리뷰 클릭
                    search_button.click()
                    time.sleep(3)

                    # 좋은점 확인하고 저장하기
                    results = find_save_좋은점(results)
                    print(f'{spot} 검색 완료')

                else:
                    print('"리뷰 탭"이 없어서 다음 관광지로 넘어갑니다.')

            else:
                print('"해당 관광지"가 없어서 다음 관광지로 넘어갑니다.')

        else:
            print('"지도 탭"이 없어서 다음 관광지로 넘어갑니다.')

    except:
        print('오류로 실패 리스트에 인덱스로 저장합니다.')
        cant_find.append(i)

driver.quit()

-------------------- 1 : 맥파이앤타이거 신사티룸 --------------------
10번째에 지도 탭있음.
서울특별시 강남구 논현로153길 44 클레어스서울 2F 맥파이앤타이거 신사티룸
1번째에 해당 관광지 있음.
5번째에 리뷰 탭있음.
"이런 점이 좋았어요" 있음.
1번째 버튼 클릭 성공
2번째 버튼 클릭 성공
3번째 버튼 클릭 성공
4번째 버튼 클릭 성공
5번째 버튼 클릭 성공
6번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
----- 1번째 좋은 점 저장 -----
----- 2번째 좋은 점 저장 -----
----- 3번째 좋은 점 저장 -----
----- 4번째 좋은 점 저장 -----
----- 5번째 좋은 점 저장 -----
----- 6번째 좋은 점 저장 -----
----- 7번째 좋은 점 저장 -----
----- 8번째 좋은 점 저장 -----
----- 9번째 좋은 점 저장 -----
----- 10번째 좋은 점 저장 -----
----- 11번째 좋은 점 저장 -----
----- 12번째 좋은 점 저장 -----
----- 13번째 좋은 점 저장 -----
----- 14번째 좋은 점 저장 -----
----- 15번째 좋은 점 저장 -----
----- 16번째 좋은 점 저장 -----
----- 17번째 좋은 점 저장 -----
----- 18번째 좋은 점 저장 -----
----- 19번째 좋은 점 저장 -----
----- 20번째 좋은 점 저장 -----
----- 21번째 좋은 점 저장 -----
----- 22번째 좋은 점 저장 -----
----- 23번째 좋은 점 저장 -----
----- 24번째 좋은 점 저장 -----
----- 25번째 좋은 점 저장 -----
----- 26번째 좋은 점 저장 -----
----- 27번째 좋은 점 저장 -----
----- 28번째 좋은 점 저장 -----
----- 29번째 좋은 점 저장 -----
맥파이앤타이거 신사티룸 검색 완료
------

## 실행결과 만들어진 list 확인
* results, cant_find

In [ ]:
results

[{'관광지명': '맥파이앤타이거 신사티룸',
  '주소': '서울특별시 강남구 논현로153길 44 클레어스서울 2F 맥파이앤타이거 신사티룸',
  '목록': '카페,디저트',
  '리뷰내용': '"차가 맛있어요"',
  '리뷰수': '654',
  '참여인원수': '862명'},
 {'관광지명': '맥파이앤타이거 신사티룸',
  '주소': '서울특별시 강남구 논현로153길 44 클레어스서울 2F 맥파이앤타이거 신사티룸',
  '목록': '카페,디저트',
  '리뷰내용': '"친절해요"',
  '리뷰수': '459',
  '참여인원수': '862명'},
 {'관광지명': '맥파이앤타이거 신사티룸',
  '주소': '서울특별시 강남구 논현로153길 44 클레어스서울 2F 맥파이앤타이거 신사티룸',
  '목록': '카페,디저트',
  '리뷰내용': '"인테리어가 멋져요"',
  '리뷰수': '415',
  '참여인원수': '862명'},
 {'관광지명': '맥파이앤타이거 신사티룸',
  '주소': '서울특별시 강남구 논현로153길 44 클레어스서울 2F 맥파이앤타이거 신사티룸',
  '목록': '카페,디저트',
  '리뷰내용': '"특별한 메뉴가 있어요"',
  '리뷰수': '347',
  '참여인원수': '862명'},
 {'관광지명': '맥파이앤타이거 신사티룸',
  '주소': '서울특별시 강남구 논현로153길 44 클레어스서울 2F 맥파이앤타이거 신사티룸',
  '목록': '카페,디저트',
  '리뷰내용': '"매장이 청결해요"',
  '리뷰수': '328',
  '참여인원수': '862명'},
 {'관광지명': '맥파이앤타이거 신사티룸',
  '주소': '서울특별시 강남구 논현로153길 44 클레어스서울 2F 맥파이앤타이거 신사티룸',
  '목록': '카페,디저트',
  '리뷰내용': '"디저트가 맛있어요"',
  '리뷰수': '308',
  '참여인원수': '862명'},
 {'관광지명': '맥파이앤타이거 신사티룸',
  '주소': '서울특별시 강남구 논

In [ ]:
cant_find

[]

# DF 만들기

In [ ]:
# 함수로 만들기
def makingDF(result, error, df):
    # 데이터 프레임 만들어주기
    # 성공, 리뷰없는 것 같이 있는 DF
    results_raw = pd.DataFrame(result)
    results_df = results_raw.copy()

    # 리뷰 내용이 있는 관광지만 모은 것을 성공 DF로 저장, 없는 것을 다시 DF로 저장
    condition = results_df.리뷰내용.isnull()

    # 성공 DF
    success = results_df[~condition].reset_index(drop=True)
    # 다시 DF
    again = results_df[condition].reset_index(drop=True)

    # 에러난 데이터만 있는
    if error != []:
        # error = df.iloc[error, :].drop(columns=['tag']).reset_index(drop=True)
        error = df.iloc[error, :].reset_index(drop=True)

    return success, again, error

In [ ]:
# 함수 적용
success_df, again_df, error_df = makingDF(results, cant_find, df)

## 실행결과 만들어진 DF 확인

In [ ]:
# 확인
# success_df
# again_df
# error_df

,관광지명,주소,리뷰내용,리뷰수,참여인원수
